In [76]:
import requests
import json
import pandas as pd
from IPython.core.debugger import set_trace ## put in the code to debug
# Date format MM/DD/YYYY
# Site DK=20 FD=2
DATE = "08/05/2019"
SITE = "20"
# Vars -- DATE, SITE
# This one can be used to get the player salaries
SLATES = "https://resultsdb-api.rotogrinders.com/api/slates?start=DATE&site=SITE"
# Vars -- SLATEID
CONTEST_OWNERSHIP = "https://resultsdb-api.rotogrinders.com/api/contest-ownership?_slateId=SLATEID"
# Vars -- CONTESTID, INDEXNUM
CONTEST_LINEUPS = "https://resultsdb-api.rotogrinders.com/api/entries?_contestId=CONTESTID&sortBy=points&order=desc&index=INDEXNUM&maxFinish=58750&players=&users=false&username=&isLive=false&minPoints=&maxSalaryUsed=&incomplete=false&positionsRemaining=&requiredStartingPositions="
# Vars SLATEID
CONTEST_SLATE = "https://resultsdb-api.rotogrinders.com/api/contests?slates=SLATEID&lean=true"
# Vars -- SLATEID (get summary)
SUMMARY = ""

In [3]:
"""
    pulls the different slates based on the date and the site
    date- date of the event 
    site- defaults to Draftkings=20, Fanduel=2
    sport- filters the slate by sport, defaults to None
    slate_type- fitlers on the type of contest, Classic by default
"""
sport_mapping = {"football": 1, "baseball": 2, "basketball": 3}
def get_slates(date, site="20", sport=None, slate_type="Classic"):
    slate = SLATES.replace("DATE", date).replace("SITE", site)
    slate_data = requests.get(slate).json()
    if sport is not None:
        slate_data = [slate for slate in slate_data if slate['sport'] == sport_mapping[sport]]
    if slate_type is not None:
        slate_data = [slate for slate in slate_data if slate['slateTypeName'] == slate_type]
    return slate_data

In [4]:
"""
    Fetches the ownerships for a particular slate. Returns the data as a dictionary of players 
    with individual contest ownership after
    slate_id - id of the slate found with the get_slates function
    returns daily_ownerships dict
"""
def get_contest_ownerships(slate_id):
    contest_ownership_url = CONTEST_OWNERSHIP.replace("SLATEID", slate_id)
    daily_ownerships = requests.get(contest_ownership_url).json()
    return daily_ownerships

In [5]:
"""
    Fetches the contests from the slate on that day.
    slate_id -- id of the slate to find contests for
    returns contests in that slate
"""
def get_contests_from_slate(slate_id):
    contests_in_slate_url = CONTEST_SLATE.replace("SLATEID", slate_id)
    contests_in_slate = requests.get(contests_in_slate_url).json()
    return contests_in_slate

In [6]:
"""
    Fetches the lineups 25 at a time based on the contest_id and index passed
    contest_id- id of the contest to fetch (found in the summary call)
    index- index of the lineups to fetch (fetches number 25*(index) - 25*(index+1))
    returns lineups
"""
def get_contest_lineups(contest_id, index=0):
    contest_lineup_url = CONTEST_LINEUPS.replace("CONTESTID", contest_id)
    daily_lineups = requests.get(contest_lineup_url).json()
    return daily_lineups

In [112]:
def export_tiered_contest_lineups():
    pass
def export_contest_lineups(date):
    import itertools
    resultsdb_date = date.strftime("%m/%d/%Y")
    date = date.strftime("%Y-%m-%d")
    daily_slate = get_slates(resultsdb_date, SITE, sport="baseball")
    slate_dataframes = []
    for slate in daily_slate:
        slate_id = slate['_id']
        contests_in_slate = get_contests_from_slate(slate_id)
        tiers = get_tiers(contests_in_slate)
        for tier in range(0, len(tiers)):
            lineups_in_tier = get_lineups_in_tier(tiers[tier], date, tier)
            lineups_in_tier = list(itertools.chain.from_iterable(lineups_in_tier))
            columns = ['username', 'points', 'contest_rank',
                       'P1', 'P2', 'C', '1B', '2B', '3B', 'SS',
                       'OF1', 'OF2', 'OF3', 'date', 'contest_id', 'tier']
            df = pd.DataFrame(lineups_in_tier, columns=columns)
            df = df.drop_duplicates()
            slate_dataframes.append(df)
    return slate_dataframes

def get_lineups_in_tier(tier, date, tier_level):    
    lineups_in_tier = []
    for contest in tier:
        max_intervals = min(20, int(contest[1] / 25) + (contest[1] % 25 > 0))
        for interval in range(0, max_intervals):
            contest_lineups = get_contest_lineups(contest[0], interval)
            if len(contest_lineups) == 0:
                continue
            entries = contest_lineups['entries']
            created_rows = create_rows(entries, date, contest[0], tier_level)
            lineups_in_tier.append(created_rows)
    return lineups_in_tier

def create_rows(lineups, date, contest_id, tier):
    rows = []
    for i in range(0, len(lineups)):
        rows.append([lineups[i]['siteScreenName'], lineups[i]['points'], lineups[i]['rank']] +
                     parse_positions(lineups[i]['lineup']) + [date, contest_id, tier])
    return rows
        
def get_value(entry, pos, num):
    try:
        return entry[pos][num]['name']
    except:
        return ""

def parse_positions(entry):
    try:
        return [get_value(entry, "P", 0), get_value(entry, "P", 1), get_value(entry, "C", 0),
                get_value(entry, "1B", 0), get_value(entry, "2B", 0), get_value(entry, "3B", 0),
                get_value(entry, "SS", 0), get_value(entry, "OF", 0), get_value(entry, "OF", 1),
                get_value(entry, "OF", 2)]
    except IndexError:
        print(entry)

def export_tiered_contest_ownerships(daily_slate):
    for slate in daily_slate:
        SLATEID = slate['_id']
        # skip slates where gameCount < 5
        if slate.get('gameCount', 0) < 5:
            continue
        # Get the names of the contests in the slate
        contests_in_slate = get_contests_from_slate(SLATEID)
        contest_names = [contest['name'] for contest in contests_in_slate]
        # Fetch daily ownerships
        daily_ownerships = get_contest_ownerships(SLATEID)
        # Parse data into a list to add to dataframe
        appended_player_data = []
        for player in daily_ownerships:
            player_data = daily_ownerships[player]
            player_row = [player_data.get('name', None), player_data.get('slatePosition', None),
                          player_data.get('salary', None), player_data.get('actualFpts', 0),
                          player_data.get('projectedFpts', 0), player_data.get('projectedOwnership', 0),
                          round(player_data.get('combinedOwnership', 0), 4)]
            player_contest_row = calculate_tiers(contests_in_slate)
            joined_list = player_row + player_contest_row + [date] + [SLATEID]
            appended_player_data.append(joined_list)
        # Create table structure from the data
        df = pd.DataFrame(appended_player_data, columns=['name',
         'slatePosition','salary', 'actualFpts', 'projectedFpts', 'projectedOwnership',
         'combinedOwnership', 'low_tier', 'mid_tier', 'high_tier', 'double_ups', 'date', 'slate_id'])
        df.to_csv('../data/resultsdb/TierContests/2017/{}-ownerships-{}.csv'.format(date, SLATEID))
        
def get_tiers(contests_in_slate):
    double_ups = filter((lambda x: "Double Up" in x['name']), contests_in_slate)
    
    double_ups = [(contest['_id'], contest['maxEntries']) for contest in double_ups]
    low_tier = [(contest['_id'], contest['maxEntries']) for contest in contests_in_slate if contest['entryFee'] <= 7]
    mid_tier = [(contest['_id'], contest['maxEntries']) for contest in contests_in_slate if (contest['entryFee'] > 7 and contest['entryFee'] < 25)]
    high_tier = [(contest['_id'], contest['maxEntries']) for contest in contests_in_slate if contest['entryFee'] >= 25]

    low_tier = [x for x in low_tier if x not in double_ups]
    mid_tier = [x for x in mid_tier if x not in double_ups]
    high_tier = [x for x in high_tier if x not in double_ups]
    return [double_ups, low_tier, mid_tier, high_tier]
    
def calculate_tiers(contests_in_slate):
    double_ups = filter((lambda x: "Double Up" in x['name']), contests_in_slate)
    double_ups = [contest['name'] for contest in double_ups]

    low_tier = [contest['name'] for contest in contests_in_slate if contest['entryFee'] <= 7]
    mid_tier = [contest['name'] for contest in contests_in_slate if (contest['entryFee'] > 7 and contest['entryFee'] < 25)]
    high_tier = [contest['name'] for contest in contests_in_slate if contest['entryFee'] >= 25]

    low_tier = [x for x in low_tier if x not in double_ups]
    mid_tier = [x for x in mid_tier if x not in double_ups]
    high_tier = [x for x in high_tier if x not in double_ups]
    
    low_tier_ownerships = [player_data.get(contest_name, 0) for contest_name in low_tier]
    mid_tier_ownerships = [player_data.get(contest_name, 0) for contest_name in mid_tier]
    high_tier_ownerships = [player_data.get(contest_name, 0) for contest_name in high_tier]
    double_ups_ownerships = [player_data.get(contest_name, 0) for contest_name in double_ups]
    
    if len(low_tier_ownerships) != 0:
        average_low_tier = round(sum(low_tier_ownerships)/len(low_tier_ownerships), 4)
    else:
        average_low_tier = 0
    if len(mid_tier_ownerships) != 0:
        average_mid_tier = round(sum(mid_tier_ownerships)/len(mid_tier_ownerships), 4)
    else:
        average_mid_tier = 0
    if len(high_tier_ownerships) != 0:
        average_high_tier = round(sum(high_tier_ownerships)/len(high_tier_ownerships), 4)
    else:
        average_high_tier = 0
    if len(double_ups_ownerships) != 0:
        average_double_ups = round(sum(double_ups_ownerships)/len(double_ups_ownerships), 4)
    else:
        average_double_ups = 0
        
    return [average_low_tier, average_mid_tier, average_high_tier, average_double_ups]

In [113]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
start_date = date(2017, 7, 16)
end_date = date(2017, 10, 2)
# start_date = date(2018, 3, 29)
# end_date = date(2018, 10, 1)
# start_date = date(2019, 3, 28)
# end_date = date(2019, 10, 1)

In [308]:
for single_date in daterange(start_date, end_date):
    resultsdb_date = single_date.strftime("%m/%d/%Y")
    date = single_date.strftime("%Y-%m-%d")
    daily_slate = get_slates(resultsdb_date, SITE, sport="baseball"
    export_tiered_contest_ownerships(daily_slate)

In [ ]:
from datetime import timedelta, date
from random import random

tiers = ['double_ups', 'low_tier', 'mid_tier', 'high_tier']
for single_date in daterange(start_date, end_date):
    date = single_date.strftime("%Y-%m-%d")
    result = export_contest_lineups(single_date)
    for res in range(0, len(result)):
        print(result[res])
        if len(result[res]) == 0:
            continue
        else:
            result[res].to_csv('../data/resultsdb/TierLineups/2017/{}-lineups-{}.csv'.format(date, random()))

Empty DataFrame
Columns: [username, points, contest_rank, P1, P2, C, 1B, 2B, 3B, SS, OF1, OF2, OF3, date, contest_id, tier]
Index: []
          username  points  contest_rank             P1               P2  \
0       floridaj20  169.40             1  Trevor Cahill      David Price   
1       levireimer  168.25             2    David Price  Masahiro Tanaka   
2     RoyalGrandpa  167.40             3  Trevor Cahill      David Price   
3         Nise0501  167.40             3  Trevor Cahill      David Price   
4        kdentures  164.40             5  Trevor Cahill      David Price   
5           gazmon  164.40             5  Trevor Cahill      David Price   
6     Malickiville  162.40             7  Trevor Cahill      David Price   
7        mindymark  162.40             7  Trevor Cahill      David Price   
8       floridaj20  160.40             9  Trevor Cahill      David Price   
9           gfay63  160.40             9  Trevor Cahill      David Price   
10          gfay63  160.40    

                 username  points  contest_rank                 P1  \
0           fantasybender  221.30             1         Mike Fiers   
1                dandydon  217.90             2       Jaime Garcia   
2                dandydon  208.90             3       Jaime Garcia   
3                  mycaps  199.25             4       Tanner Roark   
4              MrMactober  196.45             5         Mike Fiers   
5        Benjaminbutton02  190.45             6      Rick Porcello   
6                 Karr1mj  190.35             7          Rich Hill   
7                  smadog  186.25             8       Tanner Roark   
8                kpeach17  184.20             9          Rich Hill   
9                 enorlin  184.15            10       Jaime Garcia   
10               wjgrimes  183.50            11       Jaime Garcia   
11              Jizetta77  179.90            12       Jaime Garcia   
12               jchay287  179.90            12       Jaime Garcia   
13               you

Empty DataFrame
Columns: [username, points, contest_rank, P1, P2, C, 1B, 2B, 3B, SS, OF1, OF2, OF3, date, contest_id, tier]
Index: []
               username  points  contest_rank           P1             P2  \
0              chad1529  135.85             1  Josh Tomlin  Jake Odorizzi   
1          bigchris1801  135.85             1  Josh Tomlin  Jake Odorizzi   
2             bshields3  135.85             1  Josh Tomlin  Jake Odorizzi   
3          kingdavid187  134.85             4  Josh Tomlin  Jake Odorizzi   
4             beverhart  132.85             5  Josh Tomlin  Jake Odorizzi   
5                tdg913  132.85             5  Josh Tomlin  Jake Odorizzi   
6             dboss99ny  132.85             5  Josh Tomlin  Jake Odorizzi   
7             Thespaz17  132.85             5  Josh Tomlin  Jake Odorizzi   
8              stevesax  132.85             5  Josh Tomlin  Jake Odorizzi   
9            scarycards  132.85             5  Josh Tomlin  Jake Odorizzi   
10           munson

               username  points  contest_rank                 P1  \
0             mavngoose  189.40             1       Zack Wheeler   
1             mavngoose  185.30             2        Josh Tomlin   
2                marvis  184.95             3      Jake Odorizzi   
3               Pguin41  182.95             4      Jake Odorizzi   
4            magneto999  181.75             5         Jon Lester   
5            thomcotton  179.35             6    Adam Wainwright   
6           BucknellMom  178.30             7         Jon Lester   
7              suidmach  177.95             8      Jake Odorizzi   
8            NotMyMoney  176.30             9         Jon Lester   
9            justajoe2u  175.30            10         Jon Lester   
10             Hulanglo  174.35            11         Jon Lester   
11               ocdobv  173.90            12      Jake Odorizzi   
12             laspinma  173.35            13         Jon Lester   
13             oldzcool  173.30            14   

Empty DataFrame
Columns: [username, points, contest_rank, P1, P2, C, 1B, 2B, 3B, SS, OF1, OF2, OF3, date, contest_id, tier]
Index: []
          username  points  contest_rank              P1              P2  \
0       ur1ndanger  156.25             1     Chris Smith  Mike Clevinger   
1           kratom  154.25             2     Chris Smith  Mike Clevinger   
2        paulmarks  154.25             2     Chris Smith  Mike Clevinger   
3           kowalr  152.90             4     Blake Snell  Mike Clevinger   
4     Malickiville  149.70             5   Edwin Jackson     Chris Smith   
5    PITTIMMORTALS  149.25             6     Chris Smith  Mike Clevinger   
6          Luiso22  149.25             6     Chris Smith  Mike Clevinger   
7       brewcrew44  147.25             8     Chris Smith  Mike Clevinger   
8         rick4ss8  147.25             8     Chris Smith  Mike Clevinger   
9         gw007msn  147.25             8     Chris Smith  Mike Clevinger   
10         Paisano  147.25    

...

Empty DataFrame
Columns: [username, points, contest_rank, P1, P2, C, 1B, 2B, 3B, SS, OF1, OF2, OF3, date, contest_id, tier]
Index: []
          username  points  contest_rank          P1            P2  \
0           gfay63  173.50             1  Jon Lester  Chris Flexen   
1         gtdallas  170.15             2  Jon Lester  Luis Perdomo   
2       floridaj20  164.15             3  Jon Lester  Luis Perdomo   
3         moklovin  161.50             4  Jon Lester  Chris Flexen   
4       floridaj20  161.15             5  Jon Lester  Luis Perdomo   
5         Chonburi  159.15             6  Jon Lester  Luis Perdomo   
6         ximib32d  158.50             7  Jon Lester  Chris Flexen   
7         youdacao  157.15             8  Jon Lester  Luis Perdomo   
8           gfay63  155.50             9  Jon Lester  Chris Flexen   
9          Awesemo  155.50             9  Jon Lester  Chris Flexen   
10        bobalouy  155.20            11  Jon Lester  Mike Pelfrey   
11        MickeyAO  155.15

                  username  points  contest_rank            P1  \
0                 shambles  225.10             1  Trevor Bauer   
1               princessJ1  210.10             2  Trevor Bauer   
2             mikemike2302  208.50             3  Trevor Bauer   
3              packmanmatt  202.10             4  Trevor Bauer   
4                 dannyc45  201.10             5  Trevor Bauer   
5           invertedcheese  200.10             6  Trevor Bauer   
6                easton136  199.30             7  J.C. Ramirez   
7               wslymonroe  196.10             8  Trevor Bauer   
8               OnYourBike  193.10             9  Trevor Bauer   
9                 goudsaab  191.75            10  Trevor Bauer   
10             DoctorRoBry  191.10            11  Trevor Bauer   
11             Rotosoldier  190.75            12  Trevor Bauer   
12              princessJ1  190.10            13  Trevor Bauer   
13                dannyc45  190.10            13  Trevor Bauer   
14        

Empty DataFrame
Columns: [username, points, contest_rank, P1, P2, C, 1B, 2B, 3B, SS, OF1, OF2, OF3, date, contest_id, tier]
Index: []
            username  points  contest_rank             P1              P2  \
0             taze34  162.30             1   Jaime Garcia   Ariel Miranda   
1             bartg1  153.30             2   Jaime Garcia   Ariel Miranda   
2   nannerspoolparty  151.05             3      Alex Wood     Travis Wood   
3          Meabrat67  149.50             4   Jaime Garcia     Travis Wood   
4          Blanton04  147.35             5   Jaime Garcia       Alex Wood   
5          bjjball44  145.55             6      Alex Wood       Chad Kuhl   
6        sheetrockrn  145.30             7   Jaime Garcia   Ariel Miranda   
7           dezucca1  145.30             7   Jaime Garcia   Ariel Miranda   
8       perryhickman  145.00             9  Ariel Miranda     Travis Wood   
9          Bevis1834  145.00             9   Jaime Garcia       Chad Kuhl   
10        Falconett

Empty DataFrame
Columns: [username, points, contest_rank, P1, P2, C, 1B, 2B, 3B, SS, OF1, OF2, OF3, date, contest_id, tier]
Index: []
Empty DataFrame
Columns: [username, points, contest_rank, P1, P2, C, 1B, 2B, 3B, SS, OF1, OF2, OF3, date, contest_id, tier]
Index: []
                username  points  contest_rank               P1  \
0                Awesemo  137.95             1        Ivan Nova   
1              kdkjay730  128.95             2        Ivan Nova   
2                zsavior  128.75             3      Chris Smith   
3                   CZYX  125.95             4        Ivan Nova   
4               dramos77  125.75             5  Adalberto Mejia   
5                Awesemo  124.95             6        Ivan Nova   
6           OvertimeKing  124.75             7  Adalberto Mejia   
7                   CZYX  124.75             7      Chris Smith   
9               Pokey777  122.75            10  Adalberto Mejia   
10              TimFloyd  122.75            10      Chris Smit

Empty DataFrame
Columns: [username, points, contest_rank, P1, P2, C, 1B, 2B, 3B, SS, OF1, OF2, OF3, date, contest_id, tier]
Index: []
Empty DataFrame
Columns: [username, points, contest_rank, P1, P2, C, 1B, 2B, 3B, SS, OF1, OF2, OF3, date, contest_id, tier]
Index: []
              username  points  contest_rank               P1  \
0          jdeluca3838  131.80             1    Collin McHugh   
1              dlee907  127.85             2    Collin McHugh   
2            ferdaBOYS  126.90             3  Yovani Gallardo   
3               gfay63  123.90             4  Yovani Gallardo   
4               kcgeno  122.05             5     Matthew Boyd   
5        sosacorkingco  119.90             6  Yovani Gallardo   
6              Awesemo  119.00             7     Matthew Boyd   
7              mush096  119.00             7     Matthew Boyd   
8               kew707  118.05             9    Collin McHugh   
9               dl7481  117.80            10    Collin McHugh   
10              g

Lineups

Salaries

Lines

In [ ]:
# Random snippets
# daily_slate = get_slates(DATE, SITE, sport="baseball")
# SLATEID = daily_slate[1]['_id']
# daily_contest_lineups = get_contest_lineups(CONTEST_ID)
# contests_in_slate = get_contests_from_slate(SLATEID)
# contests_in_slate[0].keys()
# CONTEST_ID = "5d7f7a67d0585a501d5c53be"
# daily_ownerships = get_contest_ownerships(SLATEID)
# daily_ownerships.keys()
# contests_in_slate[0]
# daily_ownerships['Anthony Rizzo']
# daily_ownerships.keys()
# # daily slates have line data in them
# daily_slate[1]
# contests_in_slate = get_contests_from_slate("5ae5150eb819ea733461bafe")
# tiers = get_tiers(contests_in_slate)
# lineups = get_lineups_in_tier(tiers[1], "jhkh")
# list(itertools.chain.from_iterable(lineups))